In [6]:
import import_ipynb
import torch
import dataloader
import generate_mahalanobis
import regression_mahalanobis
import generate_odin
import calmetric
from models.resnet import ResNet34, ResNet18
from models.densenet import DenseNet3
import os
import random
import numpy as np
import csv

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## No Fault Injection

In [8]:
# id_datasets = ['cifar10', 'cifar100']
# ood_dataset = 'svhn'
# model_names = ['resnet34', 'densenet3']
# batch_size = 128
# magnitude = 0.0014
# temperature = 1000

# for id_dataset in id_datasets:
#     if id_dataset == 'cifar10':
#         num_classes = 10
#     elif id_dataset == 'cifar100':
#         num_classes = 100
        
#     for model_name in model_names:
#         if model_name == 'resnet34':
#             model = ResNet34(num_c=num_classes).to(device)
#         elif model_name == 'densenet3':
#             model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
#         model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
#         model.load_state_dict(torch.load(model_path, weights_only=True))
#         model.eval()
        
#         mean, std = dataloader.get_mean_std(id_dataset)    
        
#         id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
#         _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
#         ##### ODIN #####
        
#         file_path = f'./softmax_scores/{model_name}_{id_dataset}'
        
#         if not os.path.exists(file_path):
#             os.makedirs(file_path)
            
#         generate_odin.odin(model, id_testloader, out_testloader, magnitude, temperature, std, file_path)
#         calmetric.metric(model_name, id_dataset, ood_dataset, file_path)
        
#         ##### Mahalanobis #####
            
#         file_path = f'./output/{model_name}_{id_dataset}'
        
#         if not os.path.exists(file_path):
#             os.makedirs(file_path)
        
#         # generate_mahalanobis.mahalanobis(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path)
#         # regression_mahalanobis.regression(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')
        

## Fault Injection - ODIN

In [9]:
# # import fault_injector as fi
# import PytorchFS.FS as FS

# id_datasets = ['cifar10']
# ood_dataset = 'svhn'
# model_names = ['resnet34']
# batch_size = 128
# magnitude = 0.0014
# temperature = 1000
# ash_method = 'ash_p@90'

# flip_pos = ''

# first_forward_fi = True
# backward_fi = True
# second_forward_fi = True

# single_flip = True
# multi_flip = False
# burst_flip = False

# if single_flip:
#     flip_pos += '_single'
# elif multi_flip:
#     flip_pos += '_multi'    
# elif burst_flip:
#     flip_pos += '_burst'

# if first_forward_fi:
#     flip_pos += '_ff'
# if backward_fi:
#     flip_pos += '_b'
# if second_forward_fi:
#     flip_pos += '_sf'    

# for id_dataset in id_datasets:
#     if id_dataset == 'cifar10':
#         num_classes = 10
#     elif id_dataset == 'cifar100':
#         num_classes = 100
        
#     for model_name in model_names:
#         if model_name == 'resnet34':
#             model = ResNet34(num_c=num_classes).to(device)
#         elif model_name == 'densenet3':
#             model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
#         model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
#         model.load_state_dict(torch.load(model_path))
#         model.eval()
        
#         fs = FS.FS()
#         fs.setLayerInfo(model)
        
#         mean, std = dataloader.get_mean_std(id_dataset)    
        
#         id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
#         _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
#         ##### ODIN #####
        
#         file_path = f'./softmax_scores/{model_name}_{id_dataset}'
        
#         if not os.path.exists(file_path):
#             os.makedirs(file_path)
            
#         generate_odin.odin(model, id_testloader, out_testloader, magnitude, temperature, std, file_path)

#         # CSV 파일 생성 및 헤더 작성 
#         results_file = f'{file_path}/fi_results_{model_name}_{id_dataset}{flip_pos}.csv' 

#         with open(results_file, 'w', newline='') as csvfile: 
#             csv_writer = csv.writer(csvfile) 
#             if single_flip:
#                 csv_writer.writerow(['layer_name', 'bit_position', 'auroc', 'fpr', 'auroc_fi', 'fpr_fi'])  
#             elif multi_flip:
#                 csv_writer.writerow(['layer_name', 'num_flips', 'bits', 'auroc', 'fpr', 'auroc_fi', 'fpr_fi'])                  
#             elif burst_flip:
#                 csv_writer.writerow(['layer_name', 'num_flips', 'start_bit', 'auroc', 'fpr', 'auroc_fi', 'fpr_fi'])                                  
        
#         # 컨볼루션 레이어와 FC 레이어 추출
#         # layer_names = fi.get_layer_name(model)
#         all_module_names = fs.getModuleNameList(model)
#         layer_names = []
        
#         for name in all_module_names:
#             if any(keyword.lower() in name.lower() for keyword in ['conv', 'fc', 'linear']):
#                 layer_names.append(name)
        
#         # 각 컨볼루션 레이어 테스트 
#         for layer_idx, layer_name in enumerate(layer_names): 
#             if 'fc' in layer_name or 'linear' in layer_name:
#                 if id_dataset == 'cifar10':
#                     num_flips = 1
#                 elif id_dataset == 'cifar100':
#                     num_flips = 10
#             else:
#                 if single_flip:
#                     num_flips = 100
#                 else:
#                     num_flips = 100
                
#             print(f"Testing layer {layer_idx+1}/{len(layer_names)}: {layer_name}")  
            
#             layer_results = []  
            
#             # 각 비트 위치별 테스트
#             if single_flip:
#                 bit_position = 30
#                 bit_positions = [bit_position]
#                 print(f"  Testing bit position: {bit_position}")
                    
#                 generate_odin.odin_fi_ash(model, id_testloader, out_testloader, magnitude, temperature, std, file_path, 
#                                     first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
#                                     bit_lists=bit_positions, flip_ratio=num_flips, layer_name=layer_name, fs=fs, ash_method=ash_method)
                    
#                 auroc, fpr, fi_auroc, fi_fpr = calmetric.metric_ash(model_name, id_dataset, ood_dataset, file_path)

#                 generate_odin.odin_fi(model, id_testloader, out_testloader, magnitude, temperature, std, file_path, 
#                                     first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
#                                     bit_lists=bit_positions, flip_ratio=num_flips, layer_name=layer_name, fs=fs)
                    
#                 auroc, fpr, fi_auroc, fi_fpr = calmetric.metric(model_name, id_dataset, ood_dataset, file_path)
                    
#                 # 결과를 CSV에 저장 
#                 with open(results_file, 'a', newline='') as csvfile: 
#                     csv_writer = csv.writer(csvfile) 
#                     csv_writer.writerow([layer_name, bit_position, auroc, fpr, fi_auroc, fi_fpr])
#             elif multi_flip:
#                 # 1) 직접 지정한 burst 위치 리스트들
#                 bit_positions = [[10, 30], [10, 20, 30]]

#                 for bit_position in bit_positions:
#                     generate_odin.odin_fi_ash(model, id_testloader, out_testloader, magnitude, temperature, std, file_path,
#                                           first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
#                                           bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs, ash_method=ash_method)

#                     # 4) 결과 측정
#                     auroc, fpr, fi_auroc, fi_fpr = calmetric.metric_ash(model_name, id_dataset, ood_dataset, file_path)

#                     generate_odin.odin_fi(model, id_testloader, out_testloader, magnitude, temperature, std, file_path,
#                                           first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
#                                           bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs)

#                     auroc, fpr, fi_auroc, fi_fpr = calmetric.metric(model_name, id_dataset, ood_dataset, file_path)
                    
#                     with open(results_file, 'a', newline='') as csvfile:
#                         csv.writer(csvfile).writerow([layer_name, 2, bit_position, auroc, fpr, fi_auroc, fi_fpr])
#             elif burst_flip:
#                 # 1) 직접 지정한 burst 위치 리스트들
#                 bit_positions = [[30, 31], [27, 28, 29, 30]]

#                 for bit_position in bit_positions:
#                     generate_odin.odin_fi_ash(model, id_testloader, out_testloader, magnitude, temperature, std, file_path,
#                                           first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
#                                           bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs, ash_method=ash_method)

#                     # 4) 결과 측정
#                     auroc, fpr, fi_auroc, fi_fpr = calmetric.metric_ash(model_name, id_dataset, ood_dataset, file_path)

#                     generate_odin.odin_fi(model, id_testloader, out_testloader, magnitude, temperature, std, file_path,
#                                           first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
#                                           bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs)

#                     auroc, fpr, fi_auroc, fi_fpr = calmetric.metric(model_name, id_dataset, ood_dataset, file_path)

#                     # 5) CSV에 기록
#                     with open(results_file, 'a', newline='') as csvfile:
#                         csv.writer(csvfile).writerow([layer_name, len(bit_position), bit_position[0], auroc, fpr, fi_auroc, fi_fpr])    
        

## Fault Injection - Mahalanobis

In [10]:
# import fault_injector as fi
import PytorchFS.FS as FS

id_datasets = ['cifar10']
ood_dataset = 'svhn'
model_names = ['resnet34']
batch_size = 128
magnitude = 0.0014
temperature = 1000
ash_method = 'ash_p@90'

flip_pos = ''

first_forward_fi = True
backward_fi = True
second_forward_fi = True

single_flip = False
multi_flip = True
burst_flip = False

if single_flip:
    flip_pos += '_single'
elif multi_flip:
    flip_pos += '_multi'    
elif burst_flip:
    flip_pos += '_burst'

if first_forward_fi:
    flip_pos += '_ff'
if backward_fi:
    flip_pos += '_b'
if second_forward_fi:
    flip_pos += '_sf'    

for id_dataset in id_datasets:
    if id_dataset == 'cifar10':
        num_classes = 10
    elif id_dataset == 'cifar100':
        num_classes = 100
        
    for model_name in model_names:
        if model_name == 'resnet34':
            model = ResNet34(num_c=num_classes).to(device)
        elif model_name == 'densenet3':
            model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
        model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
        model.load_state_dict(torch.load(model_path))
        model.eval()
        
        fs = FS.FS()
        fs.setLayerInfo(model)
        
        mean, std = dataloader.get_mean_std(id_dataset)    
        
        id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
        _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
        ##### Mahalanobis #####
        
        file_path = f'./output/{model_name}_{id_dataset}'
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)
            
        generate_mahalanobis.mahalanobis(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path)
        auroc, fpr = regression_mahalanobis.regression(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')

        # CSV 파일 생성 및 헤더 작성 
        results_file = f'{file_path}/fi_results_{model_name}_{id_dataset}{flip_pos}.csv' 

        with open(results_file, 'w', newline='') as csvfile: 
            csv_writer = csv.writer(csvfile) 
            if single_flip:
                csv_writer.writerow(['layer_name', 'bit_position', 'auroc', 'fpr', 'auroc_fi', 'fpr_fi'])  
            elif multi_flip:
                csv_writer.writerow(['layer_name', 'num_flips', 'bits', 'auroc', 'fpr', 'auroc_fi', 'fpr_fi'])                  
            elif burst_flip:
                csv_writer.writerow(['layer_name', 'num_flips', 'start_bit', 'auroc', 'fpr', 'auroc_fi', 'fpr_fi'])                                  
        
        # 컨볼루션 레이어와 FC 레이어 추출
        # layer_names = fi.get_layer_name(model)
        all_module_names = fs.getModuleNameList(model)
        layer_names = []
        
        for name in all_module_names:
            if any(keyword.lower() in name.lower() for keyword in ['conv', 'fc', 'linear']):
                layer_names.append(name)
        
        # 각 컨볼루션 레이어 테스트 
        for layer_idx, layer_name in enumerate(layer_names): 
            if 'fc' in layer_name or 'linear' in layer_name:
                if id_dataset == 'cifar10':
                    num_flips = 1
                elif id_dataset == 'cifar100':
                    num_flips = 10
            else:
                if single_flip:
                    num_flips = 100
                else:
                    num_flips = 100
                
            print(f"Testing layer {layer_idx+1}/{len(layer_names)}: {layer_name}")  
            
            layer_results = []  
            
            # 각 비트 위치별 테스트
            if single_flip:
                bit_position = 30
                bit_positions = [bit_position]
                print(f"  Testing bit position: {bit_position}")
                    
                generate_mahalanobis.mahalanobis_fi(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path,
                                                    first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi, 
                                                    bit_lists=bit_positions, flip_ratio=num_flips, layer_name=layer_name, fs=fs)
                
                fi_auroc, fi_fpr = regression_mahalanobis.regression_fi(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')
                
                # generate_mahalanobis.mahalanobis_fi_ash(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path,
                #                                     first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi, 
                #                                     bit_lists=bit_positions, flip_ratio=num_flips, layer_name=layer_name, fs=fs, ash_method=ash_method)
                
                # fi_auroc, fi_fpr = regression_mahalanobis.regression_fi_ash(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')                
                    
                # 결과를 CSV에 저장 
                with open(results_file, 'a', newline='') as csvfile: 
                    csv_writer = csv.writer(csvfile) 
                    csv_writer.writerow([layer_name, bit_position, auroc, fpr, fi_auroc, fi_fpr])
            elif multi_flip:
                # 1) 직접 지정한 burst 위치 리스트들
                bit_positions = [[10, 30], [10, 20, 30]]

                for bit_position in bit_positions:
                    # generate_mahalanobis.mahalanobis_fi(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path,
                    #                                     first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi, 
                    #                                     bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs)
                    
                    # fi_auroc, fi_fpr = regression_mahalanobis.regression_fi(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')
                    
                    generate_mahalanobis.mahalanobis_fi_ash(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path,
                                                        first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi, 
                                                        bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs, ash_method=ash_method)
                    
                    fi_auroc, fi_fpr = regression_mahalanobis.regression_fi_ash(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')                     
                    
                    with open(results_file, 'a', newline='') as csvfile:
                        csv.writer(csvfile).writerow([layer_name, len(bit_position), bit_position, auroc, fpr, fi_auroc, fi_fpr])
            elif burst_flip:
                # 1) 직접 지정한 burst 위치 리스트들
                bit_positions = [[30, 31], [27, 28, 29, 30]]

                for bit_position in bit_positions:
                    generate_mahalanobis.mahalanobis_fi(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path,
                                                        first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi, 
                                                        bit_lists=bit_position, flip_ratio=num_flips, layer_name=layer_name, fs=fs)
                    
                    fi_auroc, fi_fpr = regression_mahalanobis.regression_fi(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')

                    # generate_mahalanobis.mahalanobis_fi_ash(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path,
                    #                                     first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi, 
                    #                                     bit_lists=bit_positions, flip_ratio=num_flips, layer_name=layer_name, fs=fs, ash_method=ash_method)
                    
                    # fi_auroc, fi_fpr = regression_mahalanobis.regression_fi_ash(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}') 
                        
                    # 5) CSV에 기록
                    with open(results_file, 'a', newline='') as csvfile:
                        csv.writer(csvfile).writerow([layer_name, len(bit_position), bit_position[0], auroc, fpr, fi_auroc, fi_fpr])      
        

num_samples: 10000
get sample mean and covariance

 Training Accuracy:(100.00%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 86.60  97.23  92.33  97.58  96.00
Input noise: Mahalanobis_0.0014

Testing layer 1/34: conv1
get sample mean and covariance

 Training Accuracy:(90.53%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
  5.11  50.75  51.97  49.88  50.81
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(90.58%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
  5.62  62.80  68.56  64.23  56.22
Input noise: Mahalanobis_0.0014

Testing layer 2/34: layer1.0.conv1
get sample mean and covariance

 Training Accuracy:(80.42%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
  6.03  66.44  64.38  64.97  60.55
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(80.47%)

get Mahalanobi

c:\Users\Soyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 79.00  95.28  90.11  94.68  93.60
Input noise: Mahalanobis_0.0014

Testing layer 29/34: layer4.0.conv2
get sample mean and covariance

 Training Accuracy:(76.96%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 79.51  95.26  90.16  94.65  93.59
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(77.02%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 78.80  95.30  90.15  94.71  93.61
Input noise: Mahalanobis_0.0014

Testing layer 30/34: layer4.1.conv1
get sample mean and covariance

 Training Accuracy:(83.91%)

get Mahalanobis scores


c:\Users\Soyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 79.34  95.35  90.21  94.78  93.67
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(84.04%)

get Mahalanobis scores


...

out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 79.24  95.34  90.16  94.93  93.63
Input noise: Mahalanobis_0.0014

Testing layer 31/34: layer4.1.conv2
get sample mean and covariance

 Training Accuracy:(83.33%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 79.14  95.32  90.14  94.73  93.64
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(83.36%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 78.96  95.30  90.21  94.68  93.63
Input noise: Mahalanobis_0.0014

Testing layer 32/34: layer4.2.conv1
get sample mean and covariance

 Training Accuracy:(82.80%)

get Mahalanobis scores


c:\Users\Soyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 78.82  95.30  90.18  94.61  93.58
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(82.64%)

get Mahalanobis scores


c:\Users\Soyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 78.78  95.26  90.12  94.52  93.54
Input noise: Mahalanobis_0.0014

Testing layer 33/34: layer4.2.conv2
get sample mean and covariance

 Training Accuracy:(81.58%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 79.20  95.32  90.16  94.87  93.61
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(81.23%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 78.91  95.33  90.11  94.89  93.62
Input noise: Mahalanobis_0.0014

Testing layer 34/34: linear
get sample mean and covariance

 Training Accuracy:(99.89%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 86.60  97.23  92.33  97.58  96.00
Input noise: Mahalanobis_0.0014

get sample mean and covariance

 Training Accuracy:(99.87%)

get Mahalanobis scores
out_distribution: svhn
 TNR    AUROC  DTACC  AUIN   AUOUT 
 86.60  97.23  92.33  97.58  96.0